# Nagel–Schreckenberg (16 points)

Nagel–Schreckenberg  is a simple model used for modelling motorway traffic. As the name suggest, it was developed by two German physicists Kai Nagel and Michael Schreckenberg in the early 1990s.
## Road
The road is divided into cells. Each cell represents a part of the road. A cell can be either empty or contains a single car (only one car in any cell).
## Car 
Cars are one cell long and have a velocity $v$, originally in range of 0-5 units per time unit.
## Transisition rules
In the transition function the following steps are performed (mind the order):
* **Acceleration**: The velocity of all cars having a velocity lower than the maximum velocity is increased by 1.
* **Slowing down**: If the distance between the current and the following car is smaller than the car's velocity, the velocity is reduced to the number of empty cells in front of the car (avoiding collisions).
* **Randomization**: The speed of all cars that have a $v > 1$ is reduced by one unit with the probability of *p*. 
* **Car motion**: All cars are moved *v* cells ahead.


Discretisation of time and space results in cellular automata. If the maximum speed is equal to 1, the model is identical to elementary cellular automaton with rule 184. 
(source and more detailed description: [here](https://en.wikipedia.org/wiki/Nagel–Schreckenberg_model))

## Basic implementation of Nagel-Schreckenberg model (8 points)

* (5 points) Create a working Nagel-Schreckenberg model using the following template, fill in the gaps. Start with randomly generate state.
* (2 points) Add periodic boundaries, i.e. cars leaving the road should appear at the beginning e.g. a car with $v=5$ located 2 cells before the end of the road should be moved to the third cell of the road.
* (1 point) Add a function that will randomly add new cars at the first cell of the road. Remember to check if the cell is empty, spawn a car only if possible. 

In [1]:
import numpy as np
import itertools
import pygame
import random
import copy

from pygame.locals import (
    K_UP,
    K_DOWN,
    K_LEFT,
    K_RIGHT,
    K_ESCAPE,
    KEYDOWN,
    QUIT,
)

pygame 2.6.1 (SDL 2.28.4, Python 3.13.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [32]:
class Car:
    
    def __init__(self, minVelocity, maxVelocity, position):
        # assign a randomly selected colour for visualisation
        self.colour = (random.randint(0,255), random.randint(0,255), random.randint(0,255))
        
        # copy parameters
        self.minVelocity = minVelocity
        self.maxVelocity = maxVelocity
        
        # generate random init velocity
        self.velocity = random.randint(minVelocity,maxVelocity)
                
        # set initial position
        self.position = position
    
    def increaseVelocity(self):
        if self.velocity < self.maxVelocity:
            self.velocity = self.velocity + 1
    
    def setVelocity(self, newVelocity): 
        if newVelocity <= self.maxVelocity:
            self.velocity = newVelocity
        
    def randomizeVelocity(self, probability):
        if random.random() < probability:
            if self.velocity > 1:
                self.velocity = self.velocity - 1
        return



In [ ]:
class Model:
    def __init__(self):
        # Parameters
        self.roadLength = 50
        self.minVelocity = 0
        self.maxVelocity = 5
        self.probabilityOfNewCar = 0
        initalNumberOfCars = 10
        
        self.time = 0
        self.states = []
        
        # Generate cars 
        initalPositions = random.sample(range(0, self.roadLength), initalNumberOfCars)
        self.cars = []
        for i in initalPositions:
            self.cars.append(Car(self.minVelocity, self.maxVelocity, i))
            
        # Sort cars arrays
        self.cars.sort(key = lambda x: x.position)
    
    def randomlyAddNewCar(self):
        if random.random() < self.probabilityOfNewCar:
            if (self.cars[0].position != 0):
                self.cars.append(Car(self.minVelocity, self.maxVelocity, 0))
        return


    
    def update(self):
        # 0. Save previous state
        self.states.append(copy.deepcopy(self.cars))
        self.time += 1

        # I. Acceleration
        for i in range(len(self.cars)):
            if self.cars[i].velocity < self.cars[i].maxVelocity:
                self.cars[i].increaseVelocity()
        
        # II. Slowing Down
        for i in range(len(self.cars)):
            distance = abs(self.cars[(i+1)%len(self.cars)].position - self.cars[i].position)
            if distance < self.cars[i].velocity:
                self.cars[i].setVelocity(distance-1)

        # III. Randomization
        for i in range(len(self.cars)):
            self.cars[i].randomizeVelocity(0.5)

        # IV. Car motion
        for i in range(len(self.cars)):
            self.cars[i].position = (self.cars[i].position + self.cars[i].velocity) % self.roadLength

        # To simplify calculations we're sorting the cars by their position
        self.randomlyAddNewCar()
        self.cars.sort(key = lambda x: x.position)
        

    def stepBack(self):
        if len(self.states) > 0:
            self.cars = self.states.pop()
            self.time -= 1
        
    def draw(self, screen, w_width, w_height):
        black = (0,0,0)
        white = (255,255,255)  
        grey = (128,128,128)
        blockSize = w_width / self.roadLength

        # draw road
        pos_x = 0
        pos_y = w_height/2
        rect = pygame.Rect(pos_x, pos_y, (blockSize)*self.roadLength, blockSize)
        pygame.draw.rect(screen, (128,128,128) , rect, 0)

    
        for x in self.cars:
            pos_x = (blockSize) * x.position
            pos_y = w_height/2
            rect = pygame.Rect(pos_x, pos_y, blockSize, blockSize)
            # Draw Cars
            pygame.draw.rect(screen, x.colour, rect, 0)
  

        pygame.display.flip()

   

In [ ]:
pygame.init()
# Parameters
w_width = 1200
w_height = 800
# Set up the drawing window, adjust the size
screen = pygame.display.set_mode([w_width, w_height])

# Create ant & grid
model = Model()

# Set background
screen.fill((255, 255, 255))

    
running = True
model.draw(screen, w_width, w_height)
while running:
    for event in pygame.event.get():   
        if event.type == QUIT:
            running = False
        
        if event.type == KEYDOWN:
            if event.key == K_RIGHT:
                model.update()
                model.draw(screen, w_width, w_height)
            elif event.key == K_LEFT:
                model.stepBack()
                model.draw(screen, w_width, w_height)


        
pygame.quit()

## Improvements (8 points)
Add:
* (1 points) Labels indicating time and number of cars.
* (2 points) Plot showing average speed vs car density (amount of cars per cell; note that it should be $<1$).
* (5 points) Create two lane motorway, implement overtaking. Car changes the lane if $v>distance$ and the left/right lane is empty (remember to check for collisions  with cars already driving the left/right lane). 
To get this points you must do a full implmenetation of basic model (including periodic boundaries and random generation). 

In [52]:
class ModelImproved:
    def __init__(self):
        # Parameters
        self.roadLength = 50
        self.minVelocity = 0
        self.maxVelocity = 5
        self.probabilityOfNewCar = 0
        self.lanes = 2
        initalNumberOfCars = 10

        self.time = 0
        self.states = []

        # Generate cars 
        initalPositions = random.sample(range(0, self.roadLength * self.lanes), initalNumberOfCars)
        self.cars = []
        for i in initalPositions:
            lane = i // self.roadLength
            position = i % self.roadLength
            self.cars.append(Car(self.minVelocity, self.maxVelocity, (position, lane)))

        # Sort cars arrays
        self.cars.sort(key=lambda x: (x.position[0], x.position[1]))

    def randomlyAddNewCar(self):
        if random.random() < self.probabilityOfNewCar:
            if all(car.position != (0, lane) for car in self.cars for lane in range(self.lanes)):
                lane = random.randint(0, self.lanes - 1)
                self.cars.append(Car(self.minVelocity, self.maxVelocity, (0, lane)))
        return

    def update(self):
        # 0. Save previous state
        self.states.append(copy.deepcopy(self.cars))
        self.time += 1

        # I. Acceleration
        for car in self.cars:
            if car.velocity < car.maxVelocity:
                car.increaseVelocity()


        # I.a Overataking
        for idx ,car in enumerate(self.cars):
            same_lane_cars = [c for c in self.cars if c.position[1] == car.position[1]]
            other_lane_cars = [c for c in self.cars if c.position[1] != car.position[1]]
            next_same_lane_car = [c for c in same_lane_cars if c.position[0] > car.position[0]][0] if len(same_lane_cars) > 1 else same_lane_cars[0]
            next_other_lane_car = [c for c in other_lane_cars if c.position[0] > car.position[0]][0] if len(other_lane_cars) > 0 else other_lane_cars[0]
            distance_same_lane = (next_same_lane_car.position[0] - car.position[0]) % self.roadLength
            distance_other_lane = (next_other_lane_car.position[0] - car.position[0]) % self.roadLength
            can_overtake = distance_same_lane < car.velocity and distance_other_lane > car.velocity
            if can_overtake:
                car.position = (car.position[0], next_other_lane_car.position[1])

        # II. Slowing Down
        for i in range(len(self.cars)):
            car = self.cars[i]
            next_car = self.cars[(i + 1) % len(self.cars)]
            distance = (next_car.position[0] - car.position[0]) % self.roadLength
            if distance < car.velocity:
                car.setVelocity(distance - 1)

        # III. Randomization
        for car in self.cars:
            car.randomizeVelocity(0.5)

        # IV. Car motion
        for car in self.cars:
            car.position = ((car.position[0] + car.velocity) % self.roadLength, car.position[1])

        # V. Overtaking


        # To simplify calculations we're sorting the cars by their position
        self.randomlyAddNewCar()
        self.cars.sort(key=lambda x: (x.position[0], x.position[1]))

    def stepBack(self):
        if len(self.states) > 0:
            self.cars = self.states.pop()
            self.time -= 1

    def draw(self, screen, w_width, w_height):
        black = (0, 0, 0)
        white = (255, 255, 255)
        grey = (128, 128, 128)
        blockSize = w_width / self.roadLength

        # draw road
        for lane in range(self.lanes):
            pos_x = 0
            pos_y = w_height / 2 + lane * blockSize
            rect = pygame.Rect(pos_x, pos_y, blockSize * self.roadLength, blockSize)
            pygame.draw.rect(screen, grey, rect, 0)

        for car in self.cars:
            pos_x = blockSize * car.position[0]
            pos_y = w_height / 2 + car.position[1] * blockSize
            rect = pygame.Rect(pos_x, pos_y, blockSize, blockSize)
            # Draw Cars
            pygame.draw.rect(screen, car.colour, rect, 0)

        # Draw labels
        font = pygame.font.Font(None, 36)
        text = font.render(f'Time: {self.time}  Cars: {len(self.cars)}', True, black)
        screen.blit(text, (10, 10))

        pygame.display.flip()

In [53]:
pygame.init()
w_width = 1200
w_height = 800
screen = pygame.display.set_mode([w_width, w_height])

model = ModelImproved()

screen.fill((255, 255, 255))

    
running = True
model.draw(screen, w_width, w_height)
while running:
    for event in pygame.event.get():   
        screen.fill((255, 255, 255))
        if event.type == QUIT:
            running = False
        
        if event.type == KEYDOWN:
            if event.key == K_RIGHT:
                model.update()
                model.draw(screen, w_width, w_height)
            elif event.key == K_LEFT:
                model.stepBack()
                model.draw(screen, w_width, w_height)


        
pygame.quit()

IndexError: list index out of range